## SIMPLEX
### Optimització - Pràctica 1

In [15]:
import numpy as np

# Función para leer el archivo y extraer las matrices
def leer_archivo(nombre_archivo):
    c = []
    A = []
    b = []

    with open(nombre_archivo, 'r') as file:
        for line in file:
            if 'c=' in line:
                while True:
                    line = next(file)
                    c.append([int(x) for x in line.split()])
                    if line=='\n':
                        c.pop()                      
                        break 
            elif 'A=' in line:
                while True:
                    line = next(file)                    
                    A.append([int(x) for x in line.split()])
                    if line=='\n':  
                        A.pop()                    
                        break 

            elif 'b=' in line:
                while True:
                    line = next(file)
                    b.append([int(x) for x in line.split()])
                    if line=='\n': 
                        b.pop()                     
                        break 
    # Convertir las listas a arrays numpy
    c = np.array(c)
    A = np.array(A)
    b = np.array(b)
    
    return c, A, b

# Nombre del archivo
nombre_archivo = 'tes.txt'

# Llamar a la función para leer el archivo
c, A, b = leer_archivo(nombre_archivo)

# Mostrar las matrices
print("Matriz c:")
print(c)
print("\nMatriz A:")
print(A)
print("\nVector b:")
print(b)

Matriz c:
[[-85 -65 -34  92 -83  93  22  76  48 -71  25 -50  79 -99   0   0   0   0
    0   0]]

Matriz A:
[[-13 -29  23 -27  57 -52 -70  81  -2  64 -54   9  68  10   0   0   0   0
    0   0]
 [ -6 -53 -37  36  85 -22  35  87  32  34  65 -64  65 -25   0   0   0   0
    0   0]
 [-13  93  31  16  45  23 -11  83  75 -65  67  10 -92  88   0   0   0   0
    0   0]
 [ 67  -8 -27 -22  47  44  -3 -23 -16  81 -24 -38 -67  24   0   0   0   0
    0   0]
 [ 80  57  77  72  52  58  53  84  77  92  57  75  68  88   1   0   0   0
    0   0]
 [-81  21 -14  73  11 -10  -7  45  95 -92  97 -79  89 -92   0  -1   0   0
    0   0]
 [ 96  -7  85 -63 -24 -95  42 -71 100  42  43 -23  83  60   0   0  -1   0
    0   0]
 [-27  92 -84  77  68  73  28  53  34 -71  59 -17 -36  -1   0   0   0   1
    0   0]
 [ -1 -28  96  74  66 -82 -24  91  63  65  64 -29  73  93   0   0   0   0
   -1   0]
 [-60 -34  21 -65  84   4  12  98 -38  -1 -99  25  21  82   0   0   0   0
    0  -1]]

Vector b:
[[ 65 232 350  35 991  55 267 2

Classe

Queda: preguntar por la lectura de los problemas, la actualización de la inversa y preguntar por la fase 1 cálculo de la SBF

generas el nuevo problema fase 1 encuentras su óptimo y la base resultante es la SBF de la siguiente parte.

SImplex

In [42]:
import numpy as np

class Simplex:
    def __init__(self, c, A, b):
        self.c = c
        self.A = A
        self.b = b
        self.m, self.n = A.shape
    def calcula(self):
        # Fase 1: Introducir variables artificiales
        c_fase1 = np.concatenate((np.zeros(self.n), np.ones(self.m)))
        A_fase1 = np.hstack((self.A, np.eye(self.m)))
        self.base = np.arange(self.n, self.n + self.m)
        self.no_base = np.arange(self.n)

        # Resolver problema auxiliar
        resolvemos_fase1 = self.SIMPLEX(A_fase1, b, c_fase1)
        if resolvemos_fase1[0] != 0:
            return "No hay solución factible"

        # Fase 2: Resolver problema original
        self.no_base = self.no_base[self.no_base < self.n]
        return self.SIMPLEX(self.A, self.b, self.c)
    def theta(self,d_b,x_b):
        menor=9999
        theta=[]
        for i,valor in enumerate(d_b):
            if valor<0:
                a=-x_b[i]/valor
                theta.append(-x_b[i]/valor)
                if a<=menor:
                        menor=a
                        indice_variable_sale=i
                else:
                    if a<menor:
                        if self.base[i]<self.base[indice_variable_sale]:
                            menor=a
                            indice_variable_sale=i
        return menor, indice_variable_sale
    def SIMPLEX(self, A, b, c):
        while True:
            self.no_base.sort()
            B = A[:, self.base]
            An = A[:, self.no_base]
            B_inv = np.linalg.inv(B)
            x_b = np.dot(B_inv, b)
            x_n=np.zeros(len(self.no_base))
            c_b = c[self.base]
            c_n = c[self.no_base]
           
            r = np.array(c_n - np.dot(np.dot(c_b, B_inv),An))
            z = np.dot(c_b, x_b)+np.dot(c_n, x_n)

            if np.all(r >= 0):
                return [z, self.base]
            
            indice_variable_entra = np.where(r < 0)[0][0]

            d_b = np.dot(-B_inv, A[:, self.no_base[indice_variable_entra]])
            if np.all(d_b >= 0):
                return "No acotado"

            theta,indice_variable_sale=self.theta(d_b,x_b)

            variable_sale = self.base[indice_variable_sale]
            self.base[indice_variable_sale] = self.no_base[indice_variable_entra]
            self.no_base[indice_variable_entra] = variable_sale


In [43]:
c= np.array([-85, -65, -34, 92, -83, 93, 22, 76, 48, -71, 25, -50, 79, -99, 0, 0, 0, 0, 0, 0])
b = np.array([65, 232, 350, 35, 991, 55, 267, 249, 520, 49])
A=np.array([[-13, -29, 23, -27, 57, -52, -70, 81, -2, 64, -54, 9, 68, 10, 0, 0, 0, 0, 0, 0],
   [-6, -53, -37, 36, 85, -22, 35, 87, 32, 34, 65, -64, 65, -25, 0, 0, 0, 0, 0, 0],
   [-13, 93, 31, 16, 45, 23, -11, 83, 75, -65, 67, 10, -92, 88, 0, 0, 0, 0, 0, 0],
   [67, -8, -27, -22, 47, 44, -3, -23, -16, 81, -24, -38, -67, 24, 0, 0, 0, 0, 0, 0],
   [80, 57, 77, 72, 52, 58, 53, 84, 77, 92, 57, 75, 68, 88, 1, 0, 0, 0, 0, 0],
   [-81, 21, -14, 73, 11, -10, -7, 45, 95, -92, 97, -79, 89, -92, 0, -1, 0, 0, 0, 0],
   [96, -7, 85, -63, -24, -95, 42, -71, 100, 42, 43, -23, 83, 60, 0, 0, -1, 0, 0, 0],
   [-27, 92, -84, 77, 68, 73, 28, 53, 34, -71, 59, -17, -36, -1, 0, 0, 0, 1, 0, 0],
   [-1, -28, 96, 74, 66, -82, -24, 91, 63, 65, 64, -29, 73, 93, 0, 0, 0, 0, -1, 0],
   [-60, -34, 21, -65, 84, 4, 12, 98, -38, -1, -99, 25, 21, 82, 0, 0, 0, 0, 0, -1]])

In [44]:
simplex = Simplex(c, A, b)
result = simplex.calcula()
print("Resultado:", result)

Resultado: [-533.9128406938213, array([11, 12,  2, 10,  0, 17,  6,  1, 16,  4])]
